# Stress Analysis in Social Media - Anchor Paper 

Shir Cohen 205790124

Guy Assa 204118616




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd 'gdrive/My Drive/AML/Final Project'
!ls

In [78]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [79]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("dreaddit-train.csv", index_col = 0)
df_test = pd.read_csv("dreaddit-test.csv", index_col = 0)
df_train.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
1,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
2,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
3,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
4,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
5,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [80]:
print(df_train.shape, df_test.shape)

(2838, 116) (715, 116)


In [81]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2838 entries, 1 to 2838
Columns: 116 entries, subreddit to sentiment
dtypes: float64(106), int64(6), object(4)
memory usage: 2.5+ MB


In [82]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 715 entries, 1 to 715
Columns: 116 entries, id to sentiment
dtypes: float64(106), int64(6), object(4)
memory usage: 653.6+ KB


In [83]:
df_train.describe()

,id,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
count,2838.000000,2838.000000,2838.000000,2.838000e+03,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,...,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000
mean,13751.999295,0.524313,0.808972,1.518107e+09,18.262156,4.684272,85.996124,35.240941,40.948231,67.044249,...,1.088001,1.120099,1.000211,1.722759,1.536400,1.879385,0.843517,9.948555,5.448836,0.040740
std,17340.161897,0.499497,0.177038,1.552209e+07,79.419166,3.316435,32.334887,26.486189,31.587117,32.880644,...,0.117159,0.085227,0.006500,0.047835,0.102971,0.058932,0.174794,21.798032,2.535829,0.195490
min,4.000000,0.000000,0.428571,1.483274e+09,0.000000,-6.620000,5.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.485400,1.200000,1.561150,0.140000,0.000000,-1.918000,-1.000000
25%,926.250000,0.000000,0.600000,1.509698e+09,2.000000,2.464243,65.000000,12.410000,12.135000,41.070000,...,1.000000,1.000000,1.000000,1.691430,1.469745,1.841782,0.750000,2.000000,3.729973,-0.072222
50%,1891.500000,1.000000,0.800000,1.517066e+09,5.000000,4.321886,81.000000,29.420000,33.520000,80.710000,...,1.000000,1.142900,1.000000,1.721430,1.530295,1.878250,0.890000,5.000000,5.210000,0.044821
75%,25473.750000,1.000000,1.000000,1.530898e+09,10.000000,6.505657,101.000000,55.057500,69.320000,96.180000,...,1.142900,1.142900,1.000000,1.751760,1.596030,1.916243,1.000000,10.000000,6.855217,0.166667
max,55757.000000,1.000000,1.000000,1.542592e+09,1435.000000,24.074231,310.000000,99.000000,99.000000,99.000000,...,1.900000,1.500000,1.200000,2.007400,2.066670,2.158490,1.000000,416.000000,21.198919,1.000000


In [84]:
# Categorical features
df_train.describe(include = 'object')

,subreddit,post_id,sentence_range,text
count,2838,2838,2838,2838
unique,10,2343,173,2820
top,ptsd,9md6qz,"(0, 5)","Hello, You are invited to complete a survey fo..."
freq,584,6,337,4


In [85]:
# Types of subreddit
df_train.subreddit.value_counts()

ptsd                584
relationships       552
anxiety             503
domesticviolence    316
assistance          289
survivorsofabuse    245
homeless            168
almosthomeless       80
stress               64
food_pantry          37
Name: subreddit, dtype: int64

In [86]:
train = df_train.drop(['post_id', 'sentence_range', 'id'], axis = 1)
test = df_test.drop(['post_id', 'sentence_range', 'id'], axis = 1)

Preprocessing 

In [87]:
# Convert lable to a numeric feature
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['subreddit'] = le.fit_transform(train['subreddit'])
train.subreddit.value_counts()

6    584
7    552
1    503
3    316
2    289
9    245
5    168
0     80
8     64
4     37
Name: subreddit, dtype: int64

Select high-correlation features according to the paper

In [88]:
train.corr().abs()['label'].sort_values(ascending = False)[:10]

label                       1.000000
lex_liwc_Tone               0.436005
lex_liwc_Clout              0.403804
lex_liwc_i                  0.400440
lex_liwc_negemo             0.387979
sentiment                   0.305157
lex_dal_min_pleasantness    0.297439
lex_liwc_Authentic          0.275282
lex_liwc_posemo             0.270672
lex_liwc_anx                0.260146
Name: label, dtype: float64

In [91]:
corr_features_04 = train.corr().abs()['label'].sort_values(ascending = False).where(lambda x : x>=0.4).dropna().keys().tolist()
corr_features_04

['label', 'lex_liwc_Tone', 'lex_liwc_Clout', 'lex_liwc_i']

Text Preprocessing

In [92]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer

stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [93]:
import re

def lower(text):
    return text.lower()

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop])

def remove_punctuation(text):
    return text.translate(str.maketrans('','', punctuation))

def clean_text(text):
    text = lower(text)
    text = remove_stopwords(text)
    text = remove_punctuation(text)
    return text

train['clean_text']=train['text'].apply(clean_text)
test['clean_text']=test['text'].apply(clean_text)

In [94]:
# Removing common words
from collections import Counter
cnt_train=Counter()
cnt_test=Counter()

for text in train['clean_text'].values:
    for word in text.split():
        cnt_train[word]+=1
        
print(cnt_train.most_common(10))

for text in test['clean_text'].values:
    for word in text.split():
        cnt_test[word]+=1
        
print(cnt_test.most_common(10))

[('im', 1352), ('like', 1180), ('know', 854), ('get', 850), ('feel', 762), ('time', 756), ('would', 753), ('me', 678), ('really', 650), ('even', 597)]
[('im', 364), ('like', 323), ('get', 233), ('feel', 204), ('know', 198), ('time', 191), ('would', 190), ('me', 177), ('really', 163), ('ive', 160)]


In [95]:
train_FREQWORDS = set([w for (w, wc) in cnt_train.most_common(10)])
test_FREQWORDS = set([w for (w, wc) in cnt_test.most_common(10)])

def remove_freqwords(text, FREQWORDS):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

train["clean_text"] = train["clean_text"].apply(lambda text: remove_freqwords(text, train_FREQWORDS))
test["clean_text"] = test["clean_text"].apply(lambda text: remove_freqwords(text, test_FREQWORDS))

In [96]:
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [97]:
lematizer=WordNetLemmatizer()

def lemmatizer_words(text):
    return " ".join([lematizer.lemmatize(word) for word in text.split()])

train['clean_text']=train['clean_text'].apply(lambda text: lemmatizer_words(text))
test['clean_text']=test['clean_text'].apply(lambda text: lemmatizer_words(text))

### Majority Baseline Implementation

In [152]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
from sklearn.metrics import *
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [153]:
class MajorityBaselineClassifier():
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.class_counter_dict = dict()
        for c in range(self.num_classes):
            self.class_counter_dict.update({c:0})
    
    def fit(self, train_x):
        for x in train_x:
            prev_value = self.class_counter_dict[x]
            self.class_counter_dict.update({x:prev_value + 1})
        best_c = -1 
        max_amount = -1
        for c in range(self.num_classes):
            if (self.class_counter_dict[c] > max_amount or best_c == -1):
                best_c = c
                max_amount = self.class_counter_dict[c]
        self.best_c = best_c
        
    def evaluate(self, test_x):
        y_pred = [self.best_c for i in range(len(test_x))]
        y_true = test_x
        accuracy = accuracy_score(y_pred, y_true)*100
        f1 = f1_score(y_pred, y_true, zero_division=0)*100
        precision = precision_score(test_labels, y_pred, zero_division=1)*100
        recall = recall_score(y_pred, y_true, zero_division=0)*100
        return accuracy, f1, precision, recall

In [154]:
train_labels = train[["label"]].to_numpy().transpose()[0]
test_labels = test[["label"]].to_numpy().transpose()[0]

model = MajorityBaselineClassifier(2)
model.fit(train_labels)

In [155]:
print('\t - Test Accuracy: {:.4f}'.format(accuracy))
print('\t - Test Precision: {:.4f}'.format(precision))
print('\t - Test Recall: {:.4f}'.format(recall))
print('\t - Test F1: {:.4f}\n'.format(f1))

	 - Test Accuracy: 51.6084
	 - Test Precision: 51.6084
	 - Test Recall: 51.6084
	 - Test F1: 68.0812



### Supervised Models with BERT Embedding



In [103]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
import tensorflow as tf
import transformers
import tqdm
from keras.preprocessing import sequence

def func_tokenizer(tokenizer_name, docs):
    features = []
    for doc in tqdm.tqdm(docs, desc = 'converting text to numerical features'):
        tokens = tokenizer_name.tokenize(doc)
        ids = tokenizer_name.convert_tokens_to_ids(tokens)
        features.append(ids)
    return features

In [105]:
# Initialize bert tokenizer
bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased') #bert-large-uncased
features = ['text'] + corr_features_04
print(features)

# Prepare the dataset with bert features
X,y = train[features], df_train['label']
bert_features = func_tokenizer(bert_tokenizer, X['text'])

['text', 'label', 'lex_liwc_Tone', 'lex_liwc_Clout', 'lex_liwc_i']


converting text to numerical features: 100%|██████████| 2838/2838 [00:12<00:00, 219.71it/s]


In [156]:
bert_trg = sequence.pad_sequences(bert_features, maxlen = 500)
X = pd.DataFrame(bert_trg)

# Add high-correlated numerical features
X = X.assign(lex_liwc_Tone= train['lex_liwc_Tone'].values)
X = X.assign(lex_liwc_Clout = train['lex_liwc_Clout'].values)
X = X.assign(lex_liwc_i = train['lex_liwc_i'].values)
X

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,lex_liwc_Tone,lex_liwc_Clout,lex_liwc_i
0,0,0,0,0,0,0,0,0,0,0,...,2031,1037,3558,1999,4466,2847,1012,1.00,15.04,9.48
1,0,0,0,0,0,0,0,0,0,0,...,2093,1002,2753,9733,1043,6169,1012,98.18,76.85,1.83
2,0,0,0,0,0,0,0,0,0,0,...,2003,1999,2010,3438,1005,1055,1012,25.77,76.38,8.98
3,0,0,0,0,0,0,0,0,0,0,...,2126,1010,1045,7977,2870,2041,1012,79.26,15.25,16.12
4,0,0,0,0,0,0,0,0,0,0,...,2438,3350,2000,2031,1037,2553,1012,1.00,28.71,7.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833,0,0,0,0,0,0,0,0,0,0,...,2673,1998,2017,2467,2097,2022,1012,99.00,99.00,5.62
2834,0,0,0,0,0,0,0,0,0,0,...,2000,2031,2026,2166,2067,2153,1012,1.00,4.45,11.11
2835,0,0,0,0,0,0,0,0,0,0,...,1010,2030,2128,15222,8950,1007,1012,80.01,97.34,0.00
2836,0,0,0,0,0,0,0,0,0,0,...,1012,2023,2003,4326,2000,2033,1012,25.77,61.58,4.41


In [141]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [205]:
def get_models():
    models = dict()
    models['SVM'] = make_pipeline(StandardScaler(),SVC())
    models['Naive Bayes'] = make_pipeline(StandardScaler(), GaussianNB())
    models['Logistic Regression'] = make_pipeline(StandardScaler(),LogisticRegression(solver = 'saga', C = 70.0))
    
    # decision trees
    models['Descision Tree'] = DecisionTreeClassifier(max_depth = 1)
    models['XGBoost'] = XGBClassifier(n_estimators = 11, max_depth = 1)
    models['Random Forest'] = RandomForestClassifier(n_estimators = 10)
    models['AdaBoost'] = AdaBoostClassifier(n_estimators= 12)
    return models

In [143]:
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
	precision_scores = cross_val_score(model, X, y, scoring='precision', cv=cv, n_jobs=-1, error_score='raise')
	recall_scores = cross_val_score(model, X, y, scoring='recall', cv=cv, n_jobs=-1, error_score='raise')
	f1_scores = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1, error_score='raise') 
	return precision_scores, recall_scores, f1_scores

In [144]:
df_results = pd.DataFrame(columns=['Model','Precision','Recall','F1-Score'])

models = get_models()
for name, model in models.items():
	precision_scores, recall_scores, f1_scores = evaluate_model(model, X, y)
	to_append = [name, round(precision_scores.mean(),4), round(recall_scores.mean(),4), round(f1_scores.mean(),4)]
	a_series = pd.Series(to_append, index = df_results.columns)
	df_results = df_results.append(a_series, ignore_index=True)

In [159]:
df_results = df_results.sort_values(by = 'F1-Score', ascending = True)
df_results

,Model,Precision,Recall,F1-Score
1,Naive Bayes,0.7697,0.0398,0.0753
6,Random Forest,0.6867,0.6043,0.6335
3,Descision Tree,0.6750,0.8123,0.7370
7,AdaBoost,0.7273,0.7554,0.7403
4,XGBoost,0.7240,0.7592,0.7406
0,SVM,0.6995,0.7989,0.7453
2,Logistic Regression,0.7244,0.7708,0.7463


In [160]:
df_results.to_csv('df_results_supervised_with_BERT_embedding.csv')

### Supervised Model with Word2Vec Embedding (Logistic Regression) 

In [161]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [162]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [163]:
!pip show gensim | grep Version

Version: 4.2.0


In [192]:
import gensim

In [193]:
# Clean data using the built in cleaner in gensim
train['text_clean'] = train['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
test['text_clean'] = test['text'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [194]:
X_train = train['text_clean']
X_test = test['text_clean'] 
y_train = train['label']
y_test = test['label']

In [195]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   #size=100,
                                   window=5,
                                   min_count=2)

In [196]:
words = set(w2v_model.wv.index_to_key) #index2word
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

In [197]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [198]:
X = pd.DataFrame(X_train_vect_avg)

# Add high-correlated numerical features
X = X.assign(lex_liwc_Tone= train['lex_liwc_Tone'].values)
X = X.assign(lex_liwc_Clout = train['lex_liwc_Clout'].values)
X = X.assign(lex_liwc_i = train['lex_liwc_i'].values)

In [199]:
models = get_models()
for name, model in models.items():
	precision_scores, recall_scores, f1_scores = evaluate_model(model, X, y_train)
	to_append = [name, round(precision_scores.mean(),4), round(recall_scores.mean(),4), round(f1_scores.mean(),4)]
	a_series = pd.Series(to_append, index = df_results.columns)
	df_results = df_results.append(a_series, ignore_index=True)

In [207]:
df_results = df_results.sort_values(by = 'F1-Score', ascending = True)
df_results

,Model,Precision,Recall,F1-Score
7,Random Forest,0.7193,0.6615,0.6943
2,Naive Bayes,0.6560,0.7621,0.7046
4,Descision Tree,0.6750,0.8123,0.7370
5,XGBoost,0.7208,0.7648,0.7413
8,AdaBoost,0.7253,0.7692,0.7458
1,SVM,0.7323,0.7891,0.7592
3,Logistic Regression,0.7496,0.7901,0.7688


In [204]:
df_results.to_csv('df_results_supervised_with_Word2vec.csv')

### Fine-tuning of pretrained BERT-base on Reddit Dataset


In [174]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [175]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [176]:
torch.cuda.empty_cache()

In [177]:
text = train.clean_text.values
labels = train.label.values

In [178]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [179]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [180]:
val_ratio = 0.2
batch_size = 128

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [181]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_f1 = tp / (tp + 0.5*(fp+fn)) if (tp + 0.5*(fp+fn)) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_f1

In [182]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained( 
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [183]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 3 # according to the paper

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_f1 = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_f1 = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update f1 only when (tn + fp) !=0; ignore nan
        if b_f1 != 'nan': val_f1.append(b_f1)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation F1: {:.4f}\n'.format(sum(val_f1)/len(val_f1)) if len(val_f1)>0 else '\t - Validation Specificity: NaN')

Epoch:  33%|███▎      | 1/3 [00:12<00:24, 12.37s/it]


	 - Train loss: 0.6738
	 - Validation Accuracy: 0.6667
	 - Validation Precision: 0.6266
	 - Validation Recall: 0.8492
	 - Validation F1: 0.7195



Epoch:  67%|██████▋   | 2/3 [00:24<00:12, 12.52s/it]


	 - Train loss: 0.5565
	 - Validation Accuracy: 0.7031
	 - Validation Precision: 0.6492
	 - Validation Recall: 0.9125
	 - Validation F1: 0.7559



Epoch: 100%|██████████| 3/3 [00:37<00:00, 12.54s/it]


	 - Train loss: 0.4082
	 - Validation Accuracy: 0.7406
	 - Validation Precision: 0.6892
	 - Validation Recall: 0.8952
	 - Validation F1: 0.7773



In [184]:
text = test.clean_text.values
labels = test.label.values

In [189]:
token_id = []
attention_masks = []

for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])

token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [190]:
batch_size = 128

val_set = TensorDataset(token_id, 
                        attention_masks, 
                        labels)

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [117]:
# Tracking variables 
val_accuracy = []
val_precision = []
val_recall = []
val_f1 = []

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      # Forward pass
      eval_output = model(b_input_ids, 
                          token_type_ids = None, 
                          attention_mask = b_input_mask)
    logits = eval_output.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    # Calculate validation metrics
    b_accuracy, b_precision, b_recall, b_f1 = b_metrics(logits, label_ids)
    val_accuracy.append(b_accuracy)
    # Update precision only when (tp + fp) !=0; ignore nan
    if b_precision != 'nan': val_precision.append(b_precision)
    # Update recall only when (tp + fn) !=0; ignore nan
    if b_recall != 'nan': val_recall.append(b_recall)
    # Update f1 only when (tn + fp) !=0; ignore nan
    if b_f1 != 'nan': val_f1.append(b_f1)

print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
print('\t - Validation F1: {:.4f}\n'.format(sum(val_f1)/len(val_f1)) if len(val_f1)>0 else '\t - Validation Specificity: NaN')


	 - Train loss: 0.3683
	 - Validation Accuracy: 0.7370
	 - Validation Precision: 0.7054
	 - Validation Recall: 0.8450
	 - Validation F1: 0.7684



References for external code repositories:
*   https://paperswithcode.com/dataset/dreaddit
*   https://www.kaggle.com/code/sohommajumder21/bert-tokenizer-with-9-models-nlp-stress-analysis
*   https://github.com/fsentin/dreaddit
*   https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381
*   https://towardsdatascience.com/fine-tuning-bert-for-text-classification-54e7df642894
*   https://mccormickml.com/2019/07/22/BERT-fine-tuning/#1-setup
